1. What is a Chain?
At its core, a Chain is an end-to-end wrapper around multiple components, executed in a specific sequence to accomplish a complex task.

Think of it like a recipe:

You don't just throw all the ingredients into a bowl at once.

You follow steps: Mix Dry Ingredients -> Cream Butter & Sugar -> Add Eggs -> Combine Wet & Dry -> Bake.

Similarly, a LangChain Chain defines a sequence of operations (often involving LLMs, but also data retrieval, code execution, etc.) to get a final result.

The core idea: Chaining different operations together, where the output of one step becomes the input to the next.

 Basic LLM Chains

In [1]:
from langchain_google_genai  import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
load_dotenv()
model=ChatGoogleGenerativeAI(model="gemini-2.5-pro")

c:\Users\anand\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


1.llmchain

In [2]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["topic"],
    template="Write a short poem about {topic}:"
)
chain = LLMChain(llm=model, prompt=prompt)
chain.invoke({"topic": "the ocean"})

C:\Users\anand\AppData\Local\Temp\ipykernel_37200\2677802900.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=model, prompt=prompt)
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 58.010233874s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimension

{'topic': 'the ocean',
 'text': 'A sapphire blanket, vast and deep,\nWhere ancient, silent secrets sleep.\nIts rhythmic breath, a constant sigh,\nBeneath the vast and open sky.\n\nIt churns in rage, it stills in peace,\nA wild heart that will never cease.\nIt holds the sun and silver moon,\nAnd hums a deep and timeless tune.'}

2.sequential chains

Sequential Chains
Run multiple chains in sequence:

SimpleSequentialChain: Single input → single output

SequentialChain: Multiple inputs → multiple outputs

A SequentialChain connects multiple chains or LLM calls in order.

In [3]:
# Step 1: Generate a product name
from langchain.chains import LLMChain, SequentialChain
prompt1 = PromptTemplate(
    input_variables=["product"],
    template="Give me a creative name for a {product} startup."
)
chain1 = LLMChain(llm=model, prompt=prompt1, output_key="startup_name")

# Step 2: Generate a tagline for that startup
prompt2 = PromptTemplate(
    input_variables=["startup_name"],
    template="Write a catchy tagline for a startup named {startup_name}."
)
chain2 = LLMChain(llm=model, prompt=prompt2, output_key="tagline")

# Combine both chains sequentially
overall_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["product"],
    output_variables=["startup_name", "tagline"],
    verbose=True
)

result = overall_chain({"product": "AI-powered language learning app"})
print(result)

C:\Users\anand\AppData\Local\Temp\ipykernel_37200\900443506.py:24: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = overall_chain({"product": "AI-powered language learning app"})




> Entering new SequentialChain chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 1.880644569s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 2
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 59.681632203s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 2
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 59
}
]

Parallel Chains in LangChain: Complete Guide
Parallel chains allow you to execute multiple chains or operations simultaneously, significantly improving efficiency for complex workflows. Let me break this down in detail.

1. Core Concepts
What are Parallel Chains?
Sequential: A → B → C (one after another)

Parallel: A, B, C executed simultaneously, then results combined

Key Benefits:
Performance: Reduce total execution time

Modularity: Independent processing of different aspects

Flexibility: Combine diverse operations

Error Isolation: One chain failing doesn't stop others

2. Implementation Approaches
A. Using RunnableParallel (LCEL - Recommended)
This is the modern, preferred approach using LangChain Expression Language.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel

load_dotenv()

# Using Gemini for both models (you can use different Gemini models if needed)
model1 = ChatGoogleGenerativeAI(model="gemini-2.5-pro")
model2 = ChatGoogleGenerativeAI(model="gemini-2.5-flash")  # Using a faster model for quiz generation

prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

parser = StrOutputParser()

# Parallel chain running both note generation and quiz generation simultaneously
parallel_chain = RunnableParallel({
    'notes': prompt1 | model1 | parser,
    'quiz': prompt2 | model2 | parser
})

# Sequential chain to merge the results
merge_chain = prompt3 | model1 | parser

# Combined chain: parallel execution followed by sequential merge
chain = parallel_chain | merge_chain

text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

result = chain.invoke({'text': text})

print("=== FINAL RESULT ===")
print(result)

print("\n=== CHAIN STRUCTURE ===")
chain.get_graph().print_ascii()

=== FINAL RESULT ===
Of course! Here are the notes and quiz merged into a single document.

***

### **Support Vector Machines (SVMs): Notes and Quiz**

#### **What are Support Vector Machines (SVMs)?**
*   A supervised learning method.
*   Used for classification, regression, and finding outliers.

#### **Advantages**
*   Works well with many features (high-dimensional data).
*   Effective even when you have more features than data samples.
*   Memory efficient because it only uses a subset of training data (called "support vectors").
*   Flexible due to the use of different "Kernel functions."

#### **Disadvantages**
*   Can overfit if you have many more features than samples. Careful tuning is required.
*   Does not give direct probability estimates. Calculating them is slow and expensive.

#### **Usage Note (scikit-learn)**
*   Can handle both dense (NumPy) and sparse (SciPy) data.

---

### **Quiz**

1.  **Q:** What are Support Vector Machines (SVMs) used for?
    **A:** SVMs are 

ImportError: Install grandalf to draw graphs: `pip install grandalf`.

Conditional Chains in LangChain
Conditional chains allow you to create dynamic workflows that route execution based on conditions, input data, or previous outputs. Here's a comprehensive guide with implementations using your existing setup.

1. Basic Conditional Routing
Using RunnableBranch (Recommended)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv()

# Using Google Gemini instead of OpenAI/Anthropic
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

parser = StrOutputParser()

class Feedback(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into positive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction': parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

# Branch chain that routes based on sentiment
branch_chain = RunnableBranch(
    (lambda x: x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x: x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "Could not determine sentiment")
)

# Complete chain: classify sentiment → branch to appropriate response
chain = classifier_chain | branch_chain

# Test the chain
print("=== TESTING FEEDBACK CHAIN WITH GEMINI ===")

test_feedbacks = [
    "This is a beautiful phone",
    "The battery life is terrible and it keeps crashing",
    "I love the camera quality and the display is amazing",
    "Worst product I've ever bought, completely disappointed"
]

for feedback in test_feedbacks:
    print(f"\n--- Feedback: '{feedback}' ---")
    result = chain.invoke({'feedback': feedback})
    print(f"Response: {result}")

# Print the chain structure
print("\n=== CHAIN STRUCTURE ===")
chain.get_graph().print_ascii()

=== TESTING FEEDBACK CHAIN WITH GEMINI ===

--- Feedback: 'This is a beautiful phone' ---


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-1.5-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..


NotFound: 404 models/gemini-1.5-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.